In [33]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import scipy
import sklearn
from sklearn.preprocessing import OneHotEncoder

In [24]:
applied=pd.read_csv('./application_record.csv')
approved=pd.read_csv('./credit_record.csv')

In [31]:
lwr=[]
for x in applied.columns:
    lwr.append(x.lower())
applied.columns=lwr
fff=applied[['code_gender', 'flag_own_car', 'flag_own_realty', 'cnt_children',
       'amt_income_total', 'name_income_type', 'name_education_type',
       'name_family_status', 'name_housing_type', 'days_birth',
       'days_employed', 'flag_mobil', 'flag_work_phone', 'flag_phone',
       'flag_email', 'occupation_type', 'cnt_fam_members']]
for f in fff.columns:
    print(fff[f].value_counts())

F    294440
M    144117
Name: code_gender, dtype: int64
N    275459
Y    163098
Name: flag_own_car, dtype: int64
Y    304074
N    134483
Name: flag_own_realty, dtype: int64
0     304071
1      88527
2      39884
3       5430
4        486
5        133
7          9
9          5
12         4
6          4
14         3
19         1
Name: cnt_children, dtype: int64
135000.0    46879
157500.0    39768
180000.0    37912
112500.0    36979
225000.0    34130
            ...  
300600.0        1
621000.0        1
109899.0        1
283185.0        1
83304.0         1
Name: amt_income_total, Length: 866, dtype: int64
Working                 226104
Commercial associate    100757
Pensioner                75493
State servant            36186
Student                     17
Name: name_income_type, dtype: int64
Secondary / secondary special    301821
Higher education                 117522
Incomplete higher                 14851
Lower secondary                    4051
Academic degree                     31

In [45]:
cats=['code_gender', 'flag_own_car', 'flag_own_realty','name_income_type', 'name_education_type','name_family_status', 'name_housing_type','occupation_type']

In [55]:
cat_cols=pd.get_dummies(fff[cats],drop_first=False)
numers=list(set(fff.columns) - set(fff[cats].columns))
fin_df=pd.concat((fff[numers],cat_cols),axis=1)
fin_df

,cnt_fam_members,flag_phone,flag_mobil,days_employed,amt_income_total,flag_work_phone,cnt_children,flag_email,days_birth,code_gender_F,...,occupation_type_Laborers,occupation_type_Low-skill Laborers,occupation_type_Managers,occupation_type_Medicine staff,occupation_type_Private service staff,occupation_type_Realty agents,occupation_type_Sales staff,occupation_type_Secretaries,occupation_type_Security staff,occupation_type_Waiters/barmen staff
0,2.0,0,1,-4542,427500.0,1,0,0,-12005,0,...,0,0,0,0,0,0,0,0,0,0
1,2.0,0,1,-4542,427500.0,1,0,0,-12005,0,...,0,0,0,0,0,0,0,0,0,0
2,2.0,0,1,-1134,112500.0,0,0,0,-21474,0,...,0,0,0,0,0,0,0,0,1,0
3,1.0,1,1,-3051,270000.0,0,0,1,-19110,1,...,0,0,0,0,0,0,1,0,0,0
4,1.0,1,1,-3051,270000.0,0,0,1,-19110,1,...,0,0,0,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
438552,1.0,0,1,365243,135000.0,0,0,0,-22717,0,...,0,0,0,0,0,0,0,0,0,0
438553,1.0,0,1,-3007,103500.0,0,0,0,-15939,1,...,1,0,0,0,0,0,0,0,0,0
438554,1.0,0,1,-372,54000.0,1,0,0,-8169,1,...,0,0,0,0,0,0,1,0,0,0
438555,2.0,0,1,365243,72000.0,0,0,0,-21673,1,...,0,0,0,0,0,0,0,0,0,0


In [13]:
approved
lowr=[]
for x in approved.columns:
    lowr.append(x.lower())
approved.columns=lowr
approved

,id,months_balance,status
0,5001711,0,X
1,5001711,-1,0
2,5001711,-2,0
3,5001711,-3,0
4,5001712,0,C
...,...,...,...
1048570,5150487,-25,C
1048571,5150487,-26,C
1048572,5150487,-27,C
1048573,5150487,-28,C
